In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# coding=utf-8
import tweepy
import re
import string
import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
pd.set_option('display.max_colwidth',1000)

In [2]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
#printmd('**bold**')

In [3]:
df = pd.read_csv("FinalListPreprocessedData.csv",encoding='utf-8', engine='python')
df = df.dropna()

In [4]:
df.head()

,Unnamed: 0,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT
0,2,1.420000e+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොයා බැලීමේ වැඩ පිලිවලක් ඕනේ,1,1433
1,3,1.420000e+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා කරමින් හැකි ඉක්මණින් ආර්ථිකය,10,13898
2,4,1.420000e+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුරෝපීය රට වලට විසා නිකුත් නොකි,0,87
3,5,1.420000e+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුර,0,87
4,6,1.420000e+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87


In [45]:
import ast
import json

In [46]:
df['LABEL'].iloc[0]

"['Crime','Politics']"

In [47]:
ast.literal_eval(df['LABEL'].iloc[0])

['Crime', 'Politics']

In [8]:
#convert all form of data into the list

df['LABEL'] = df['LABEL'].apply(lambda x: ast.literal_eval(x))
                                 

In [9]:
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words
0,2,1.410000e+18,7/11/2021 9:59,"[Crime, Politics]",අන්තර්ජාල ඔස්සේ අළෙවි වූ බාලවයස්කාර දැරිය අපයෝජන කිරීමේ සිද්ධිය සම්බන්ධ පොලිස් ක්රීඩා කොට්ඨාසයේ උප පොලිස් පරීක්ෂකවරයකු රජයේ ප්රධාන පෙළේ බැංකුවක හිටපු කළමණාකාරවරයකු ළමා කාන්තා කාර්යාංශ අද අත්අඩංගුව ගනු ලැබ තිබේ
1,4,1.410000e+18,7/11/2021 9:14,[Sport],ක්රීඩා අමාත්යාංශයෙන් බොක්සිං මිලියන
2,6,1.410000e+18,7/11/2021 9:13,[Sport],මේසපන්දුව දියුණු කරන්නට ක්රීඩා අමාත්යංශය ගිවිසුම්ගත වෙයි
3,8,1.410000e+18,7/11/2021 6:39,"[Sport, Other]",පී සරවනමුත්තු කුසලා අන්තර්සමාජ පළමු ඛාණ්ඩ ශූරයන් වූ සිංහල ක්රීඩා සමාජ පිල ශ්රී ලංකව විවිධ අවධි වල නියෝජන කල ක්රීඩකයන් රැසක් සිටින එවක ශ්රී ලංකා නායක අර්ජුන රණතුංග සාමාන් ක්රීඩකයෙක් වසරේදී ක්රීඩා සමාජයට ක්රීඩා කර ඇත
4,10,1.410000e+18,7/11/2021 6:11,[Politics],ක්රීඩා ඇමැතිතුමා තවම හරි අය හරි තැන්වල දාගෙන සුනිල ගුණවර්ධන


In [10]:
#apply onehot encoding

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from skmultilearn.adapt import MLkNN


from sklearn.multiclass import OneVsRestClassifier

In [11]:
y = df['LABEL']
y

0    [Crime, Politics]
1              [Sport]
2              [Sport]
3       [Sport, Other]
4           [Politics]
5      [Entertainment]
6              [Sport]
7              [Other]
Name: LABEL, dtype: object

In [12]:
multilabel = MultiLabelBinarizer()

In [13]:
y = multilabel.fit_transform(df['LABEL'])  

In [14]:
y

array([[1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0]])

In [15]:
multilabel.classes_

array(['Crime', 'Entertainment', 'Other', 'Politics', 'Sport'],
      dtype=object)

In [16]:
pd.DataFrame(y, columns= multilabel.classes_)

,Crime,Entertainment,Other,Politics,Sport
0,1,0,0,1,0
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,1,0,1
4,0,0,0,1,0
5,0,1,0,0,0
6,0,0,0,0,1
7,0,0,1,0,0


In [17]:
#TFIDF Vectorizer

tfidf = TfidfVectorizer(analyzer = 'word', max_features = 10000, ngram_range=(1,2))
X = tfidf.fit_transform(df['stemming_words'])

In [18]:
X

<8x169 sparse matrix of type '<class 'numpy.float64'>'
	with 187 stored elements in Compressed Sparse Row format>

In [19]:
#tfidf.vocabulary_

In [20]:
X.shape, y.shape

((8, 169), (8, 5))

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

# Build Model

In [22]:
sgd = SGDClassifier()
lr = LogisticRegression(solver ='lbfgs')
svc = LinearSVC()


In [23]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100

def print_score(y_pred, clf):
    print("clf", clf.__class__.__name__)
    print('Jaccard score:{}'.format(j_score(y_test, y_pred)))
    print('---')   
                                

In [24]:
for classifier in [sgd, lr, svc]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

clf SGDClassifier
Jaccard score:75.0
---
clf LogisticRegression
Jaccard score:0.0
---
clf LinearSVC
Jaccard score:0.0
---


# Model Test with Real Data

In [25]:
 x = ['ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය ගෙන දුන් ක්රීඩකයන් කටුනායක ගුවන්තොටුප පරිශ්රය උණුසුම පිළිගැනීම']

In [27]:
xt = tfidf.transform(x)

In [29]:
clf.predict(xt)

array([[0, 0, 0, 0, 0]])

In [30]:
multilabel.inverse_transform(clf.predict(xt)) 

[()]

In [ ]:
from skmultilearn.adapt import MLkNN

